### CS 421 PROJECT

In this project, you will be working with data extracted from famous recommender systems type datasets: you are provided with a large set of interactions between users (persons)  and items (movies). Whenever a user "interacts" with an item, it watches the movie and gives a mark or "rating" between 1 and 5 stars (5 stars indicating that the user liked that movie very much, and 1 star indicating that the user didn't like the movie at all. 




In this exercise, we will **not** be performing the recommendation task per se. Instead, we will identify *anomalous users*. In the dataset that you are provided with, some of the data was corrupted. Whilst most of the data comes from real life user-item interactions from a famous movie rating website, some "users" are anomalous: they were generated by me according to some undisclosed procedure. 

You are provided with two data frames: the first one ("ratings") contains the interactions provided to you, and the second one ("labels") contains the labels for the users.

As you can see, the three columns in "ratings" correspond to the user ID, the item ID and the rating. Thus, each row of "ratings" contains a single interaction. For instance, if the row "142, 152, 5" is present, this means that the user with ID 142 has given the movie 152 the rating 5 stars.

The dataframe "labels" has two columns. In the first column we have the user ids, whilst the second column contains the labels. A label of 1 indicates that the user is fake (generated by me), whilst a label of 0 denotes a natural user (coming from real life interactions). 

For instance, if the labels matrix contains the line "142, 1", it means that all of the ratings given by the user with id 142 are fake. This means all lines in the dataframe "ratings" which start with the userID 142 correspond to fake interactions. 

#### Evaluation

Your task is to be able to classify unseen instances as either anomalies or non anomalies (guess whether they are real users or if they were generated by me). 

There are **far more** normal users than anomalies in the dataset, which makes this a very heavily **unbalanced dataset**. Thus, accuracy will not be a good measure of performance, since simply predicting that every user is normal will give good accuracy. Thus, we need to use some other evaluation metrics (see lecture notes from week 3). 

THE **EVALUATION METRICS** are:  THE **AUC** (AREA UNDER CURVE), the **PRECISION**, THE **RECALL**, and the **F1 score**. The **main metric** will be the **AREA UNDER CURVE**, and it will by default be used to rank teams. This means your programs should return an **anomaly score** for each user (the higher the score, the more likely the model think the sample is anomalous).  

Every few weeks, we will evaluate the performance of each team (on an *unseen test set* I will provide) in terms of AUC, PRECISION, RECALL and F1 score, and rank the teams by **AUC** and by F1 score to distinguish between ties, where a tie is defined by a difference of less than 0.005 in AUC.  

The difficulty implied by **the generation procedure of the anomalies WILL CHANGE as the project evolves: depending on how well the teams are doing, I will generate easier or harder anomalies**.

The **first round** will take place after recess (week 9): this means that I will **release the next test set on the tuesday of week 9**, and you must hand in your scores before the **WEDNESDAY at NOON (5th of October)**. We will then look at the results together on the thursday.  

We will check everyone's performance in this way every week (once on  week 10, once on week 11 and once on week 12). 

Whilst performance (expressed in terms of AUC and your ranking compared to other teams) at **each of the check points** (weeks 9 to 12 inclusive) is an **important component** of your **final grade**, the **final report** and the detail of the various methods you will have tried will **also** be very **important**. Ideally, to get perfect marks (A+), you should try at least **two supervised methods** and **two unsupervised methods**, as well as be ranked the **best team** in terms of performance.

The performance part of the grading will be based half on performance at weeks 9,10,11 and half on performance at week 12. 

In [1]:
import numpy as np
import pandas as pd
data=np.load("first_batch.npz")

In [2]:
X=data["X"]
y=data["y"]


XX=pd.DataFrame(X)
yy=pd.DataFrame(y)
XX.rename(columns={0:"user",1:"item",2:"rating"},inplace=True)

In [3]:
XX.head()

,user,item,rating
0,2549,0,4
1,2549,30,3
2,2549,55,2
3,2549,67,2
4,2549,72,4


In [4]:
# Get the number of unique values in each column
XX.nunique()

user      5200
item       999
rating       6
dtype: int64

In [5]:
yy.rename(columns={0:"user",1:"label"},inplace=True)

In [6]:
yy.head(10)

,user,label
0,0,0
1,1,0
2,2,0
3,3,0
4,4,0
5,5,0
6,6,0
7,7,0
8,8,0
9,9,0


In [7]:
# Create a 2D array 5200 rows (num users), 1000 cols (num movies)
user_reviews = [[-1] * 1000] * 5200

for row in X:
  user_idx = row[0]
  movie_idx = row[1]
  rating = row[2]

  user_reviews[user_idx][movie_idx] = rating

In [9]:
# Add list of movie reviews to yy
# This will be training data for the classification model
yy["reviews"] = user_reviews
yy

# Model should output prediction array of 5200 - 1 for each user
# We can either predict 0-1 or probabilities
# Metrics are AUC, precision, recall, F1

,user,label,reviews
0,0,0,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 4, 1, 1, ..."
1,1,0,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 4, 1, 1, ..."
2,2,0,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 4, 1, 1, ..."
3,3,0,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 4, 1, 1, ..."
4,4,0,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 4, 1, 1, ..."
...,...,...,...
5195,5195,0,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 4, 1, 1, ..."
5196,5196,0,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 4, 1, 1, ..."
5197,5197,0,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 4, 1, 1, ..."
5198,5198,0,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 4, 1, 1, ..."
